# Budget Analytics: How Expensive Is Life in Israel?

## Table of Contents

1. [Project description](#project-description)
1. [Imports](#Imports)  
2. [Data Loading](#Data-Loading)  
3. [Initial Data Overview](#Initial-Data-Overview)  
4. [Preprocessing](#Preprocessing)  
5. [Overall Spending Profile](#Overall-Spending-Profile)  
6. [Conclusions](#Conclusions)

## Project description

### Objective:
A two-year spending analysis for a family of three, focused on the following:
- Overall and dynamic expenditure (total, average, median)
- Spending structure and internal distribution across categories and subcategories
- Comparison of individual consumption with official statistics

### Observation period:
June 2023 – May 2025

### Description:
This project is based on real daily expenses of a family of three during their first two years living in Israel after repatriation.

### Family composition (age at the time of repatriation):
- Andrey — 43 years old
- Ekaterina — 36 years old
- Agata — 1 year old

### Location:
Kiryat Ono

### Data description:
- **`budget.csv`** — primary data file 
- **`exchange_rates_2023_2025.json`** —  historical USD/NIS exchange rates for the observation period, used for currency conversion

Fields in `budget.csv`:  
- `id` — purchase ID (int64)
- `purchase_name` — item name (object)
- `purchase_subcategory` — subcategory of the item (object)
- `price` — amount in NIS (float64)
- `financing_source` — payment method (card, cash, bank check) (object)
- `purchase_date` — date and time of transaction (datetime64)
- `buyers_name` — name of the person who entered the transaction (object)
- `purchase_category` — high-level category of the purchase (object)

### Data limitations:

The recorded purchase timestamps may be inaccurate. Since the Telegram bot runs locally, data is saved at the moment the bot is launched, not at the actual time of purchase.

The buyer name is derived from Telegram message metadata and may not match the actual purchaser — for example, if one family member entered data on behalf of another. The current version of the bot does not support manual name input.

**As a result**, the fields `purchase_date` and `buyers_name` are not used as primary variables in the analysis.
However, for certain categories such as `Personal` and `Food`, the recorded buyer name is typically reliable, as these purchases are made individually.
For other categories — such as `Taxes`, `Utilities`, or other shared family expenses — the buyer name reflects the person who entered the data, not necessarily the actual purchaser.
Buyer-related analysis is therefore limited and only applied where attribution is reliable.

### Currency:

All monetary values are shown in Israeli Shekels (NIS) unless stated otherwise. A price_usd column is available for reference, but price_nis is used as the primary currency in this notebook.

## Imports

In [1]:
# Standard library imports
import os
import sys

# Third-party imports
import pandas as pd

# Add src to sys.path for local imports from src/utils.py
sys.path.append(os.path.abspath(os.path.join(os.pardir, "src")))  # <-- Added for local imports
import utils  # <-- Local imports

## Data Loading

Load the main dataset (e.g. CSV) containing transaction records.

In [2]:
# Define the path to the dataset
BUDGET_PATH = "../data/budget.csv"
RATES_PATH = "../data/exchange_rates_2023_2025.json"

# Function to parse dates with UTC timezone
DATE_FORMAT = lambda x: pd.to_datetime(x, utc=True)

# Load main dataset
budget = pd.read_csv(
    BUDGET_PATH,
    converters={"purchase_date": DATE_FORMAT}
)

# Load exchange rates from JSON
exchange_rates = pd.read_json(RATES_PATH).T.reset_index()
exchange_rates.rename(
    columns={
        "index": "exchange_date",
        "USDILS": "rate_usd_to_nis"
    },
    inplace=True)

## Initial Data Overview

Check general structure, column types, and sample data.

In [3]:
# Display first rows
budget.head()

,id,purchase_name,purchase_subcategory,price,financing_source,purchase_date,buyers_name,purchase_category
0,2068,kindergarten,Daycare,1050.00,Card,2025-05-25 08:14:00+00:00,Ekaterina,Kids
1,2067,Health insurance,Health and Beauty,235.51,Card,2025-05-21 08:12:00+00:00,Ekaterina,Personal
2,2066,Bus Pass,Transport,315.00,Card,2025-05-14 08:08:00+00:00,Ekaterina,Transport
3,2065,SIM card,Telecoms,40.70,Card,2025-05-23 08:04:00+00:00,Ekaterina,Personal
4,2064,Hotel,Leisure and entertainment,1944.55,Card,2025-05-23 08:03:00+00:00,Ekaterina,Personal


In [4]:
budget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2006 entries, 0 to 2005
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   id                    2006 non-null   int64              
 1   purchase_name         2006 non-null   object             
 2   purchase_subcategory  2006 non-null   object             
 3   price                 2006 non-null   float64            
 4   financing_source      2006 non-null   object             
 5   purchase_date         2006 non-null   datetime64[ns, UTC]
 6   buyers_name           2006 non-null   object             
 7   purchase_category     2006 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(1), object(5)
memory usage: 125.5+ KB


In [5]:
exchange_rates.head()

,exchange_date,rate_usd_to_nis
0,2023-06-01,3.745050
1,2023-06-02,3.749060
2,2023-06-03,3.749060
3,2023-06-04,3.749365
4,2023-06-05,3.740185


In [6]:
exchange_rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   exchange_date    731 non-null    datetime64[ns]
 1   rate_usd_to_nis  731 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 11.6 KB


>## Initial Data Overview Summary
>
>The dataset includes 2006 rows and 8 columns describing household purchases, with appropriate data types and no missing values. Text fields contain case and spacing inconsistencies, which are addressed during preprocessing.
>
>In addition, a separate exchange rate dataset covering the USD/NIS rate from 2023-06-01 to 2025-05-31 has been loaded. It contains daily rates and is prepared for merging with the main dataset to enable conversion of prices to USD.
>
>The data is ready for preprocessing.

## Preprocessing

- Normalize categorical and text fields

All columns with `object` dtype are to be converted to lowercase to ensure uniform formatting across the dataset.
This step is sufficient for all categorical fields (`purchase_category`, `purchase_subcategory`, `financing_source`, `buyers_name`), as they already follow a consistent structure.
The `purchase_name` column is also converted to lowercase, but no further normalization is applied, since this field is not involved in the current analysis.

In [7]:
# Mask for all object-type (text) columns
mask_object = budget.select_dtypes(include="object")

# Normalize text: lowercase and strip spaces
budget.loc[:, mask_object.columns] = mask_object.apply(
    lambda col: col.str.lower().str.strip()
)
budget.head()

,id,purchase_name,purchase_subcategory,price,financing_source,purchase_date,buyers_name,purchase_category
0,2068,kindergarten,daycare,1050.00,card,2025-05-25 08:14:00+00:00,ekaterina,kids
1,2067,health insurance,health and beauty,235.51,card,2025-05-21 08:12:00+00:00,ekaterina,personal
2,2066,bus pass,transport,315.00,card,2025-05-14 08:08:00+00:00,ekaterina,transport
3,2065,sim card,telecoms,40.70,card,2025-05-23 08:04:00+00:00,ekaterina,personal
4,2064,hotel,leisure and entertainment,1944.55,card,2025-05-23 08:03:00+00:00,ekaterina,personal



- Check the actual date range in the data to ensure it matches the defined observation period

In [8]:
min_date = budget["purchase_date"].min()
max_date = budget["purchase_date"].max()

print(f"Min date: {min_date}\nMax date: {max_date}")

Min date: 2023-06-07 21:00:00+00:00
Max date: 2025-06-05 07:56:00+00:00


We have some rows with purchase dates after May 2025. These rows fall outside the defined observation period and should be removed.
To simplify filtering and support further analysis, we will create `purchase_year` and `purchase_month` columns.
This allows for cleaner year-over-year and month-over-month comparisons without applying *.dt.year* or *.dt.month* in every query.

In [9]:
budget["purchase_year"] = budget["purchase_date"].dt.year
budget["purchase_month"] = budget["purchase_date"].dt.month

# Delete all rows with purchases made in June 2025
mask_june = (budget["purchase_date"].dt.month == 6) & (budget["purchase_date"].dt.year == 2025)
budget = budget[~mask_june].copy()

# Refresh the mask and check tyhe result
mask_june = (budget["purchase_date"].dt.month == 6) & (budget["purchase_date"].dt.year == 2025)
budget[mask_june]

,id,purchase_name,purchase_subcategory,price,financing_source,purchase_date,buyers_name,purchase_category,purchase_year,purchase_month


- Find and process duplicates

We’ll begin by identifying full duplicates based on all available fields except the `id` field — even though such exact matches are unlikely, they could still indicate mistakes during data entry or database operations.

After that, we’ll truncate `purchase_date` to the nearest hour and check for suspiciously similar entries. If we observe patterns or clusters that suggest accidental duplication, we’ll decide how to handle them accordingly.

In [10]:
col_duplicate = budget.columns.to_list()
col_duplicate.remove('id')
col_duplicate

['purchase_name',
 'purchase_subcategory',
 'price',
 'financing_source',
 'purchase_date',
 'buyers_name',
 'purchase_category',
 'purchase_year',
 'purchase_month']

In [11]:
print(f"Duplicates found: {budget[col_duplicate].duplicated().sum()}")

Duplicates found: 3


In [12]:
# Check duplicated rows
budget[budget[col_duplicate].duplicated(keep=False)].sort_values(by=col_duplicate)

,id,purchase_name,purchase_subcategory,price,financing_source,purchase_date,buyers_name,purchase_category,purchase_year,purchase_month
1940,76,building maintenance fee,housing,250.0,bank,2023-07-02 21:00:00+00:00,andrey,home,2023,7
1941,75,building maintenance fee,housing,250.0,bank,2023-07-02 21:00:00+00:00,andrey,home,2023,7
1831,185,chocolate,sweets and pastries,5.0,card,2023-08-13 21:00:00+00:00,andrey,food,2023,8
1833,183,chocolate,sweets and pastries,5.0,card,2023-08-13 21:00:00+00:00,andrey,food,2023,8
1561,455,sweets,sweets and pastries,6.9,card,2023-12-03 03:05:00+00:00,andrey,food,2023,12
1562,454,sweets,sweets and pastries,6.9,card,2023-12-03 03:05:00+00:00,andrey,food,2023,12


Three potentially duplicate entries were identified, all dated within 2023. Two of them belong to the first two months of bot operation, during which timestamp recording had reduced precision — only date, hour, and minute were stored.

Additionally, if the bot was offline, the timestamp for all delayed entries would be identical at the moment of processing, making matching timestamps an expected outcome.

Considering the low monetary amounts, the limited number of such cases, and — most importantly — the inability to reliably confirm duplication, the *Chocolate* and *Sweets* purchases will not be treated as duplicates.

The *Building maintenance fee* entries, however, will be verified as a recurring monthly expense. The number of such entries should correspond to the number of observation months.

In [13]:
mask_fee = (budget["purchase_name"] == "building maintenance fee")
n_entries = budget[mask_fee]["id"].count()
print(f"Number of 'building maintenance fee' entries: {n_entries}")

Number of 'building maintenance fee' entries: 22


In [14]:
min_date = budget["purchase_date"].min()
max_date = budget["purchase_date"].max()

# Calculate number of months (inclusive)
n_months = (max_date.year - min_date.year) * 12 + (max_date.month - min_date.month) + 1
print(f"Observed months: {n_months}")

Observed months: 24


The number of *"building maintenance fee"* entries (22) is **less than** the number of observed months (24).

Therefore, we can conclude that:
- The duplicate entry for *"building maintenance fee"* in July 2023 is likely a manual entry error rather than a true duplicate.
- There are two months with missing entries, assuming the fee is paid once per month.

In [15]:
budget[mask_fee][["purchase_year", "purchase_month"]].sort_values(by=["purchase_year", "purchase_month"])

,purchase_year,purchase_month
1940,2023,7
1941,2023,7
1864,2023,8
1775,2023,9
1609,2023,11
1453,2024,1
1429,2024,2
1364,2024,3
1284,2024,4
1192,2024,5


The values were sorted by year and month to visually identify duplicate and missing entries within the expected monthly pattern. As a result, we conclude that one of the two entries for *July* 2023 likely corresponds to *June* 2023 and was entered with an incorrect month. Additionally, entries are missing for *October* and *December* 2023.

In [16]:
# Correct manually entered date: this row was misdated as July instead of June
budget.loc[1940, ["purchase_date", "purchase_month"]] = ["2023-06-02 21:00:00+00:00", 6]

In [17]:
# Add missing recurring entries for "building maintenance fee"
target_months = {(2023, 10), (2023, 12)}
default_values = {
    "purchase_name": "building maintenance fee",
    "purchase_subcategory": "housing",
    "price": 250.0,
    "financing_source": "bank",
    "buyers_name": "andrey",
    "purchase_category": "home"
}
budget = utils.add_missing_recurring(budget, "building maintenance fee", target_months, default_values)
# Check the result
mask_fee = (budget["purchase_name"] == "building maintenance fee")
budget[mask_fee][["purchase_year", "purchase_month"]].sort_values(by=["purchase_year", "purchase_month"])

,purchase_year,purchase_month
1939,2023,6
1940,2023,7
1863,2023,8
1774,2023,9
2005,2023,10
1608,2023,11
2006,2023,12
1452,2024,1
1428,2024,2
1363,2024,3


Two new "building maintenance fee" entries were added for October and December 2023 to fill observed gaps.  
`id` values were adjusted to follow the existing sequence, and the updated table confirms chronological completeness.

- Recurring Payments Consistency Check

The analysis of missing entries for `building maintenance fee` highlighted the need to verify other recurring payments as well.

In this step, we will check for completeness of records corresponding to fixed-period payments, specifically for:
`apartment rent`, `property tax (arnona)`, `water supply`, `electricity`, and `gas`.

Recurring payments related to `subscriptions` will not be included in this automated check due to their non-fixed nature — they are often paused, cancelled, or renewed manually.

In [18]:
# List of recurring purchase names to check
recurring_payments_names = [
    "apartment rent",
    "property tax (arnona)",
    "water supply",
    "electricity",
    "gas"
]

# Count number of entries for each recurring item
recurring_counts = (
    budget[budget["purchase_name"].isin(recurring_payments_names)]
    .groupby("purchase_name")
    .size()
    .sort_values(ascending=False)
)

display(recurring_counts)

purchase_name
apartment rent           25
water supply             12
electricity              11
gas                      10
property tax (arnona)     6
dtype: int64

A preliminary count of entries for recurring payments shows the following:

- **Apartment rent** — 25 entries. This includes the standard monthly payments and an additional entry likely corresponding to a deposit or insurance (e.g., for potential damage or early termination), which is typical and not an anomaly.

- **Water supply** — 12 entries. This aligns with the expected bi-monthly billing cycle and requires no further review.

- **Electricity** — 11 entries. Although slightly fewer than water, this is still considered normal. The missing 12th entry corresponds to June 2025, which is outside the dataset's observation window.

- **Property tax (arnona)** — 6 entries. This payment is typically made in lump sums or irregular installments, so no validation is necessary.

- **Gas** — 10 entries. Given its expected bi-monthly pattern, this number may indicate missing data. A closer inspection will follow.

In [19]:
mask_gas = (budget["purchase_name"] == "gas")
budget[mask_gas][["purchase_year", "purchase_month"]].sort_values(by=["purchase_year", "purchase_month"])

,purchase_year,purchase_month
1269,2023,6
1906,2023,8
1753,2023,10
1584,2023,12
1377,2024,2
1187,2024,4
1029,2024,6
670,2024,8
620,2024,10
222,2025,2


In [20]:
# Add missing recurring entries for "gas"
target_months = {(2024, 12), (2025, 4)}
default_values = {
    "purchase_name": "gas",
    "purchase_subcategory": "utilities",
    "price": 0.0,  # Will be updated below
    "financing_source": "card",
    "buyers_name": "andrey",
    "purchase_category": "home"
}
# For each missing month, set the correct price before calling the utility function
gas_prices = {(2024, 12): 58.42, (2025, 4): 61.38}
for ym, price in gas_prices.items():
    if ym in target_months:
        default_values["price"] = price
        budget = utils.add_missing_recurring(budget, "gas", {ym}, default_values)
# Check the result
mask_gas = (budget["purchase_name"] == "gas")
budget[mask_gas][["purchase_year", "purchase_month"]].sort_values(by=["purchase_year", "purchase_month"])

,purchase_year,purchase_month
1269,2023,6
1906,2023,8
1753,2023,10
1584,2023,12
1377,2024,2
1187,2024,4
1029,2024,6
670,2024,8
620,2024,10
2007,2024,12


- Standardize placeholder values

The value `"Undefined"` in the `purchase_category` and `purchase_subcategory` fields indicates that the purchase name has not yet been mapped to a category or subcategory.
This is expected behavior when a new purchase name appears for the first time. Such cases are handled manually.

In [21]:
len(budget.query('purchase_category == "undefined"'))

15

Auto-fill categories for known purchase names. For purchase names with undefined category/subcategory, assign a known (non-undefined) pair if it is uniquely associated with that name.

In [22]:
# Mask for rows where category or subcategory is undefined
mask_undef = (budget["purchase_category"] == "undefined") | (budget["purchase_subcategory"] == "undefined")

# Get unique purchase names with undefined classification
undef_names = budget.loc[mask_undef, "purchase_name"].unique()

for name in undef_names:
    # Mask for all rows with the same purchase name
    mask_all = budget["purchase_name"] == name

    # Subset where category and subcategory are already defined
    mask_defined = mask_all & (budget["purchase_category"] != "undefined") & (budget["purchase_subcategory"] != "undefined")

    # Get all unique (category, subcategory) pairs for this name
    defined_categories = budget.loc[mask_defined, ["purchase_category", "purchase_subcategory"]].drop_duplicates()

    # If only one unique (category, subcategory) pair is found — assign it
    if len(defined_categories) == 1:
        category, subcategory = defined_categories.iloc[0]
        budget.loc[mask_all & mask_undef, ["purchase_category", "purchase_subcategory"]] = category, subcategory


In [23]:
undef_names = budget.loc[mask_undef, "purchase_name"].unique()
len(budget.query("purchase_category == 'undefined'"))

0

Basic normalization and rule-based filling have been applied to improve classification coverage in `purchase_category` and `purchase_subcategory`.
While many artifacts may still remain in the `purchase_name` field (such as inconsistent formatting or spelling variants), this field is not used directly in the analysis.
Therefore, full normalization of `purchase_name` is not currently necessary.

- Standardize compound purchase names by replacing punctuation with "and"

In [24]:
# Check if there are any compound purchase names separated with comma
comma_mask = (budget["purchase_name"].str.contains(","))
budget[comma_mask]["purchase_name"]

1441    bread, water
Name: purchase_name, dtype: object

In [25]:
# Replace commas with "and" to ensure consistency in compound names
budget.loc[comma_mask, "purchase_name"] = budget.loc[comma_mask, "purchase_name"].str.replace(r"\s*,\s*", " and ", regex=True)
budget.iloc[1441]

id                                            574
purchase_name                     bread and water
purchase_subcategory          sweets and pastries
price                                        40.0
financing_source                             card
purchase_date           2024-02-02 13:51:00+00:00
buyers_name                             ekaterina
purchase_category                            food
purchase_year                                2024
purchase_month                                  2
Name: 1441, dtype: object

- Merge exchange rates with purchase data

Join the main dataset with daily USD/NIS rates using the purchase date as key. This enables currency conversion.

In [26]:
# Normalize purchase date to remove time and timezone for matching with exchange rate dates
budget["exchange_date"] = budget["purchase_date"].dt.normalize().dt.tz_localize(None)

# Merge exchange rates into budget data and rename 'price' to 'price_NIS'
budget = budget.merge(
    exchange_rates,
    how="left",
    on="exchange_date"
).rename(columns={"price": "price_NIS"})

# Calculate price in USD using daily exchange rate
budget["price_USD"] = round(budget["price_NIS"] / budget["rate_usd_to_nis"], 2)

# Preview the enriched dataset
budget.head()

,id,purchase_name,purchase_subcategory,price_NIS,financing_source,purchase_date,buyers_name,purchase_category,purchase_year,purchase_month,exchange_date,rate_usd_to_nis,price_USD
0,2068,kindergarten,daycare,1050.00,card,2025-05-25 08:14:00+00:00,ekaterina,kids,2025,5,2025-05-25,3.610965,290.78
1,2067,health insurance,health and beauty,235.51,card,2025-05-21 08:12:00+00:00,ekaterina,personal,2025,5,2025-05-21,3.546105,66.41
2,2066,bus pass,transport,315.00,card,2025-05-14 08:08:00+00:00,ekaterina,transport,2025,5,2025-05-14,3.543215,88.90
3,2065,sim card,telecoms,40.70,card,2025-05-23 08:04:00+00:00,ekaterina,personal,2025,5,2025-05-23,3.612975,11.26
4,2064,hotel,leisure and entertainment,1944.55,card,2025-05-23 08:03:00+00:00,ekaterina,personal,2025,5,2025-05-23,3.612975,538.21


- Reorder columns for readability

Reordering columns to group related fields and improve overall readability of the dataset.
Chronological, descriptive, and financial fields are organized to reflect the logical structure of a purchase record.

In [27]:
# Reorder columns and sort rows for logical structure and better readability
column_order = [
    "id", "purchase_date", "purchase_year", "purchase_month",
    "buyers_name", "purchase_name", "purchase_category",
    "purchase_subcategory", "price_NIS", "price_USD", "financing_source"
]

# Create a cleaned copy for EDA and preserve the original structure
budget_cleaned = budget[column_order].copy()
budget_cleaned.sort_values(by="purchase_date",
                           ascending=False,
                           ignore_index=True,
                           inplace=True)
budget_cleaned.head()

,id,purchase_date,purchase_year,purchase_month,buyers_name,purchase_name,purchase_category,purchase_subcategory,price_NIS,price_USD,financing_source
0,2029,2025-05-31 05:56:53+00:00,2025,5,andrey,breakfast,food,cafes and restaurants,31.0,8.83,card
1,2028,2025-05-31 05:56:27+00:00,2025,5,andrey,chips,food,junk food,19.0,5.41,card
2,2027,2025-05-30 11:15:55+00:00,2025,5,andrey,sprite,food,sweets and pastries,10.0,2.84,card
3,2026,2025-05-30 11:15:55+00:00,2025,5,andrey,falafel,food,fast food,23.0,6.54,card
4,2025,2025-05-30 11:14:16+00:00,2025,5,andrey,sprite,food,sweets and pastries,12.9,3.67,card


>## Preprocessing Summary
>
>All necessary data cleaning steps have been completed to prepare the dataset for analysis. Categorical and text fields were normalized to lowercase for consistency. Out-of-scope entries beyond the observation period were removed, and additional time-based columns were created for easier aggregation.
>
>Duplicate detection was carried out using both full-row matching and timestamp truncation. Although some entries appeared suspicious, only one was flagged as a likely manual error. Two missing entries for the recurring "building maintenance fee" were added based on expected monthly frequency.
>
>"Undefined" categories were reviewed and supplemented with known values where possible, using rule-based filling. Lastly, the column structure was reordered for clarity and logical coherence.
>
>The dataset is now cleaned, consistent, and ready for spending analysis.

## Overall Spending Profile

### Basic Metrics
- Total amount spent in NIS over the full observation period

In [28]:
total_spent = budget_cleaned['price_NIS'].sum()
f"{total_spent:,.2f} NIS"

'499,218.37 NIS'

- Total amount spent in NIS by purchase category

In [29]:
total_spent_by_categories = (
    budget_cleaned.groupby("purchase_category")[["price_NIS"]]
        .sum()
        .rename(columns={"price_NIS": "total_spent_NIS"})
        .sort_values("total_spent_NIS", ascending=False)
    ).reset_index()
total_spent_by_categories["part_in_total_pct"] = (total_spent_by_categories["total_spent_NIS"] / total_spent * 100).round(2)
total_spent_by_categories

,purchase_category,total_spent_NIS,part_in_total_pct
0,home,185475.47,37.15
1,food,110019.19,22.04
2,personal,94064.92,18.84
3,kids,53346.80,10.69
4,transport,50742.41,10.16
5,pets,4236.59,0.85
6,loans,1332.99,0.27


>**The first tables yield the first insight:**
>
>Spending on our 3.5-year-old child over the observed period is nearly equal to the total cost of a 2008 Subaru Impreza with 185,000 km — including the price of the car itself.

- Total number of purchases

In [30]:
total_purchases = len(budget_cleaned)
print(f"{total_purchases} purchases made over observed period")

2009 purchases made over observed period


- Number of unique categories

In [31]:
unique_cat = budget_cleaned["purchase_category"].nunique()
print(f"There are {unique_cat} unique categories in the DataFrame")

There are 7 unique categories in the DataFrame


- Number of unique subcategories

In [32]:
unique_cat = budget_cleaned["purchase_subcategory"].nunique()
print(f"There are {unique_cat} unique subcategories in the DataFrame")

There are 23 unique subcategories in the DataFrame



### Exploratory Data Analysis (EDA)

In [33]:
budget_cleaned

,id,purchase_date,purchase_year,purchase_month,buyers_name,purchase_name,purchase_category,purchase_subcategory,price_NIS,price_USD,financing_source
0,2029,2025-05-31 05:56:53+00:00,2025,5,andrey,breakfast,food,cafes and restaurants,31.0,8.83,card
1,2028,2025-05-31 05:56:27+00:00,2025,5,andrey,chips,food,junk food,19.0,5.41,card
2,2027,2025-05-30 11:15:55+00:00,2025,5,andrey,sprite,food,sweets and pastries,10.0,2.84,card
3,2026,2025-05-30 11:15:55+00:00,2025,5,andrey,falafel,food,fast food,23.0,6.54,card
4,2025,2025-05-30 11:14:16+00:00,2025,5,andrey,sprite,food,sweets and pastries,12.9,3.67,card
...,...,...,...,...,...,...,...,...,...,...,...
2004,24,2023-06-10 21:00:00+00:00,2023,6,ekaterina,market,food,groceries,35.0,9.75,cash
2005,25,2023-06-10 21:00:00+00:00,2023,6,ekaterina,fruits,food,groceries,120.0,33.43,cash
2006,19,2023-06-10 21:00:00+00:00,2023,6,ekaterina,groceries,food,groceries,3200.0,891.50,cash
2007,11,2023-06-07 21:00:00+00:00,2023,6,andrey,taxi,transport,transport,90.6,24.79,card


## Conclusions

- Summary of findings
- Insights from spending patterns
- Suggested actions (e.g. budget adjustments, category tracking)